# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model = "gemini-2.0-flash"

gpt_system = "You are third mate of the whaling ship Pequod. Your name is Flask. \
You approach the practice of whaling as if trying to avenge some deep offense the whales have done to you. \
You are chatting with Starbuck (the chief mate) and Ishmael (an oarsman)"

claude_system = "You are the chief mate of the whaling ship Pequod. Your name is Starbuck. You are a thoughtful and intellectual \
Quaker from Nantucket who considers it madness to want revenge on an animal. \
You are chatting with two other users named Flask (the third mate) and Ishmael (an oarsman)."

gemini_system = "You are an oarsman on the Pequod (a whaling ship). They call you Ishmael. You are interested in the history and mechanics \
of whaling and attempt to promote the nobility of the trade. \
You are chatting with two users named Flask (third mate) and Starbuck (the chief mate)."

gpt_messages = ["Flask: Hi there"]
claude_messages = ["Starbuck: Hi"]
gemini_messages = ["Ishmael: Ahoy"]

In [ ]:
openai = OpenAI()
claude = anthropic.Anthropic()
gemini = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [ ]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt_message, claude_message, gemini_message in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt_message})
        messages.append({"role": "user", "content": claude_message})
        messages.append({"role": "user", "content": gemini_message})
        
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
def call_claude():
    messages = []
    for gpt_message, claude_message, gemini_message in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt_message})
        messages.append({"role": "assistant", "content": claude_message})
        messages.append({"role": "user", "content": gemini_message})
        
    messages.append({"role": "user", "content": gpt_messages[-1]})
    
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [ ]:
def call_gemini():
    messages = [{"role": "system", "content": gemini_system}]
    for gpt_message, claude_message, gemini_message in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt_message})
        messages.append({"role": "user", "content": claude_message})
        messages.append({"role": "assistant", "content": gemini_message})  
    messages.append({"role": "user", "content": gpt_messages[-1]})
    messages.append({"role": "user", "content": claude_messages[-1]})

    response = gemini.chat.completions.create(
        model=gemini_model,
        messages=messages
    )
    return response.choices[0].message.content

In [ ]:
gpt_messages = ["Flask: Ahoy men"]
claude_messages = ["Starbuck: Hello"]
gemini_messages = ["Ishmael: Ahoy! Has seen the White Whale?"]

print(f"{gpt_messages[0]}\n")
print(f"{claude_messages[0]}\n")
print(f"{gemini_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Starbuck: {claude_next}\n")
    claude_messages.append(claude_next)

    gemini_next = call_gemini()
    print(f"{gemini_next}\n")
    gemini_messages.append(gemini_next)